In [1]:
#@title <b>Time Out Preventer (Advanced) </b></strong>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
#@markdown **Run this code to prevent Google Colab from Timeout**
from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True)
if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button");
      btn.click()
      }

    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect");
      btn.click()
      }
    }

  setInterval(ClickConnect,60000)
  '''))

In [2]:
import pandas as pd
import torch
from torch.optim import AdamW  # Import PyTorch's AdamW

from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
#from transformers import LongformerTokenizer, LongformerForSequenceClassification, AdamW

from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/ADL-Hallucination-Detection')

Mounted at /content/drive


In [4]:
# Load the datasets
train_data = pd.read_csv("cnndm/train_data_base.csv", sep=';')
valid_data = pd.read_csv("cnndm/valid_data_base.csv", sep=';')
test_data = pd.read_csv("cnndm/test_data_base.csv", sep=';')

In [5]:
# print(train_data.isna().sum())
# missing_rows = train_data[train_data.isna().any(axis=1)]


In [6]:
train_data.head()

,Unnamed: 0,article,highlights,label
0,1,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,0
1,2,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,0
2,3,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",0
3,4,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",0
4,5,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",0


In [7]:

# # Load tokenizer
# tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

# # Load Longformer model for classification
# model = LongformerForSequenceClassification.from_pretrained(
#     "allenai/longformer-base-4096",
#     num_labels=2  # Binary classification
# )

# # Check model details
# print(model.config)

# This unfortunately does not work - model is too large

In [8]:
# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = BertForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=2)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Tokenize and calculate token lengths
#def calculate_lengths(data, tokenizer, column_name):
#    lengths = []
#    for text in data[column_name]:
#        tokens = tokenizer.tokenize(text)
#        lengths.append(len(tokens))
#    return lengths
#
## Calculate token lengths for documents and summaries
#doc_lengths = calculate_lengths(train_data, tokenizer, column_name='article')
#summ_lengths = calculate_lengths(train_data, tokenizer, column_name='highlights')
#
# Combine lengths for total sequence lengths
#combined_lengths = [d + s + 3 for d, s in zip(doc_lengths, summ_lengths)]  # +3 for [CLS], [SEP], [SEP]

# Create a DataFrame for analysis
#lengths_df = pd.DataFrame({
#    'Document Lengths': doc_lengths,
#    'Summary Lengths': summ_lengths,
#    'Combined Lengths': combined_lengths
#})

# Plot distributions
#plt.figure(figsize=(12, 6))
#plt.hist(lengths_df['Combined Lengths'], bins=50, alpha=0.7, label='Combined Lengths')
#plt.hist(lengths_df['Document Lengths'], bins=50, alpha=0.7, label='Document Lengths')
#plt.hist(lengths_df['Summary Lengths'], bins=50, alpha=0.7, label='Summary Lengths')
#plt.axvline(x=512, color='red', linestyle='--', label='Max Length (512)')
#plt.xlabel('Number of Tokens - Training')
#plt.ylabel('Frequency')
#plt.legend()
#plt.title('Token Length Distributions')
#plt.show()

# Print statistics
#print(lengths_df.describe())


In [10]:
# Calculate token lengths for documents and summaries
#doc_lengths_test = calculate_lengths(test_data, tokenizer, column_name='article')
#summ_lengths_test = calculate_lengths(test_data, tokenizer, column_name='highlights')

# Combine lengths for total sequence lengths
#combined_lengths_test = [d + s + 3 for d, s in zip(doc_lengths_test, summ_lengths_test)]  # +3 for [CLS], [SEP], [SEP]

# Create a DataFrame for analysis
#lengths_df_test = pd.DataFrame({
#    'Document Lengths': doc_lengths_test,
#    'Summary Lengths': summ_lengths_test,
#    'Combined Lengths': combined_lengths_test
#})

#print(lengths_df_test.describe())



In [11]:
# # Calculate token lengths for documents and summaries
# doc_lengths_valid = calculate_lengths(valid_data, tokenizer, column_name='article')
# summ_lengths_valid = calculate_lengths(valid_data, tokenizer, column_name='highlights')

# # Combine lengths for total sequence lengths
# combined_lengths_valid = [d + s + 3 for d, s in zip(doc_lengths_valid, summ_lengths_valid)]  # +3 for [CLS], [SEP], [SEP]

# # Create a DataFrame for analysis
# lengths_df_valid = pd.DataFrame({
#     'Document Lengths': doc_lengths_valid,
#     'Summary Lengths': summ_lengths_valid,
#     'Combined Lengths': combined_lengths_valid
# })

# print(lengths_df_valid.describe())

In [12]:
# print((lengths_df["Document Lengths"] > 1024).sum())
# print((lengths_df_test["Document Lengths"] > 1024).sum())
# print((lengths_df_valid["Document Lengths"] > 1024).sum())
# print()

# print((lengths_df["Document Lengths"] > 2048).sum())
# print((lengths_df_test["Document Lengths"] > 2048).sum())
# print((lengths_df_valid["Document Lengths"] > 2048).sum())

In [13]:
# print((lengths_df["Combined Lengths"] > 1024).sum())
# print((lengths_df_test["Combined Lengths"] > 1024).sum())
# print((lengths_df_valid["Combined Lengths"] > 1024).sum())
# print()

# print((lengths_df["Combined Lengths"] > 2048).sum())
# print((lengths_df_test["Combined Lengths"] > 2048).sum())
# print((lengths_df_valid["Combined Lengths"] > 2048).sum())

In [14]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.examples = []
        self.skipped_count = 0  # Counter for rows skipped due to document length
        self.skipped_bc_chunk = 0  # Counter for rows skipped due to chunking issues

        self._create_examples()

    def _create_examples(self):
        for _, row in self.data.iterrows():
            doc, summ, label = row['article'], row['highlights'], row['label']

            # Tokenize document and summary
            doc_tokens = self.tokenizer.tokenize(doc)
            summ_tokens = self.tokenizer.tokenize(summ)

            # Ensure document fits within max_length alone
            if len(doc_tokens) + 3 > self.max_length:  # [CLS] doc_tokens [SEP]
                self.skipped_count += 1
                continue

            # Function to check if a chunk fits within max_length
            def chunk_fits(tokens_chunk):
                return len(doc_tokens) + len(tokens_chunk) + 3 <= self.max_length

            # Case 1: Check if the full summary fits
            if chunk_fits(summ_tokens):
                input_ids, attention_mask = self._create_input(doc_tokens, summ_tokens)
                self.examples.append({"input_ids": input_ids, "attention_mask": attention_mask, "label": label})
            else:
                # Case 2: Try splitting into halves
                mid = len(summ_tokens) // 2
                if chunk_fits(summ_tokens[:mid]) and chunk_fits(summ_tokens[mid:]):
                    for chunk in [summ_tokens[:mid], summ_tokens[mid:]]:
                        input_ids, attention_mask = self._create_input(doc_tokens, chunk)
                        self.examples.append({"input_ids": input_ids, "attention_mask": attention_mask, "label": label})
                else:
                    # Case 3: Try splitting into thirds
                    third = len(summ_tokens) // 3
                    chunks = [summ_tokens[:third], summ_tokens[third:2 * third], summ_tokens[2 * third:]]
                    if all(chunk_fits(chunk) for chunk in chunks):
                        for chunk in chunks:
                            input_ids, attention_mask = self._create_input(doc_tokens, chunk)
                            self.examples.append({"input_ids": input_ids, "attention_mask": attention_mask, "label": label})
                    else:
                        # Case 4: Skip if none of the strategies work
                        self.skipped_bc_chunk += 1

    def _create_input(self, doc_tokens, summ_tokens):
        input_ids = [self.tokenizer.cls_token_id] + \
                    self.tokenizer.convert_tokens_to_ids(doc_tokens) + \
                    [self.tokenizer.sep_token_id] + \
                    self.tokenizer.convert_tokens_to_ids(summ_tokens) + \
                    [self.tokenizer.sep_token_id]

        attention_mask = [1] * len(input_ids)

        # Pad if necessary
        if len(input_ids) < self.max_length:
            pad_length = self.max_length - len(input_ids)
            input_ids += [self.tokenizer.pad_token_id] * pad_length
            attention_mask += [0] * pad_length

        return input_ids, attention_mask

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        return {
            "input_ids": torch.tensor(example["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(example["attention_mask"], dtype=torch.long),
            "label": torch.tensor(example["label"], dtype=torch.long)
        }


In [15]:
# wir schauen uns mal wie viele rausfliegen und wie die token lengths verteilt sind

In [16]:
# Parameters
MAX_LEN = 512 # 2048 would be better

# Prepare datasets
train_dataset = CustomDataset(train_data, tokenizer, max_length=MAX_LEN)
valid_dataset = CustomDataset(valid_data, tokenizer, max_length=MAX_LEN)
test_dataset = CustomDataset(test_data, tokenizer, max_length=MAX_LEN)


In [17]:
print(f'Train: {train_dataset.skipped_count}/{train_data.shape[0]} ~ {train_dataset.skipped_count/train_data.shape[0] * 100} %')
print(f'Test: {test_dataset.skipped_count}/{test_data.shape[0]} ~ {test_dataset.skipped_count/test_data.shape[0] * 100} %')
print(f'Valid: {valid_dataset.skipped_count}/{valid_data.shape[0]} ~ {valid_dataset.skipped_count/valid_data.shape[0] * 100} %')

Train: 10186/13952 ~ 73.00745412844036 %
Test: 1356/1986 ~ 68.27794561933534 %
Valid: 1426/1992 ~ 71.5863453815261 %


In [18]:
print(f'Train: {train_dataset.skipped_bc_chunk}/{train_data.shape[0]} ~ {train_dataset.skipped_bc_chunk/train_data.shape[0] * 100} %')
print(f'Test: {test_dataset.skipped_bc_chunk}/{test_data.shape[0]} ~ {test_dataset.skipped_bc_chunk/test_data.shape[0] * 100} %')
print(f'Valid: {valid_dataset.skipped_bc_chunk}/{valid_data.shape[0]} ~ {valid_dataset.skipped_bc_chunk/valid_data.shape[0] * 100} %')

Train: 370/13952 ~ 2.6519495412844036 %
Test: 42/1986 ~ 2.1148036253776437 %
Valid: 42/1992 ~ 2.108433734939759 %


In [19]:
BATCH_SIZE = 16

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [20]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

cpu


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
# Training function
def train_epoch(model, data_loader, optimizer, criterion, device):
    model.train()
    losses = []
    correct_predictions = 0

    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        print("model training...", end="\r")
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        print("logits computed...", end="\r")
        loss = criterion(logits, labels)
        losses.append(loss.item())
        print("loss computed...", end="\r")

        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return correct_predictions.double() / len(data_loader.dataset), sum(losses) / len(losses) # TODO hier vielleicht AUC?



In [22]:
def eval_model(model, data_loader, criterion, device):
    model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            print("model eval...", end="\r")
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)
            print("logits computed...", end="\r")
            losses.append(loss.item())
            print("loss computed...", end="\r")

            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    return correct_predictions.double() / len(data_loader.dataset), sum(losses) / len(losses)


In [23]:
EPOCHS = 20  # Maximum number of epochs
PATIENCE = 5  # Number of epochs to wait for improvement
best_val_loss = float('inf')  # Initialize with a large value
early_stopping_counter = 0  # Tracks epochs without improvement

for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    print("-" * 10)

    # Train for one epoch
    print("reached training function")
    train_acc, train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")

    # Evaluate on validation set
    print("reached eval function")
    val_acc, val_loss = eval_model(model, valid_loader, criterion, device)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

    # Check for improvement
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0  # Reset counter if performance improves
        torch.save(model.state_dict(), 'best_model_state.bin')  # Save the best model
        print("Validation performance improved. Model saved.")
    else:
        early_stopping_counter += 1  # Increment counter if no improvement
        print(f"No improvement. Early stopping counter: {early_stopping_counter}/{PATIENCE}")

    # Stop training if early stopping criteria are met
    if early_stopping_counter >= PATIENCE:
        print("Early stopping triggered. Training stopped.")
        break

# Load the best model after training
model.load_state_dict(torch.load('best_model_state.bin'))


Epoch 1/20
----------
reached training function
Train Loss: 0.6888, Train Accuracy: 0.5326
reached eval function
Validation Loss: 0.6572, Validation Accuracy: 0.7292
Validation performance improved. Model saved.
Epoch 2/20
----------
reached training function
Train Loss: 0.4187, Train Accuracy: 0.8692
reached eval function
Validation Loss: 0.2899, Validation Accuracy: 0.9070
Validation performance improved. Model saved.
Epoch 3/20
----------
reached training function
Train Loss: 0.2525, Train Accuracy: 0.9159
reached eval function
Validation Loss: 0.2614, Validation Accuracy: 0.9120
Validation performance improved. Model saved.
Epoch 4/20
----------
reached training function
Train Loss: 0.1729, Train Accuracy: 0.9439
reached eval function
Validation Loss: 0.1969, Validation Accuracy: 0.9252
Validation performance improved. Model saved.
Epoch 5/20
----------
reached training function
Train Loss: 0.1242, Train Accuracy: 0.9605
reached eval function
Validation Loss: 0.1502, Validation Acc

<ipython-input-23-22f1151cc6e0>:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_state.bin'))


<All keys matched successfully>

In [28]:
model.load_state_dict(torch.load('best_model_state.bin'))

test_acc, test_loss = eval_model(model, test_loader, criterion, device)
print(f'Test loss: {test_loss} | Test accuracy: {test_acc}')

# Classification report
all_preds = []
all_labels = []
all_probs = []

model.eval()
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, preds = torch.max(logits, dim=1)
        probs = torch.softmax(logits, dim=1)[:, 1]  # Get probabilities for the positive class

        all_probs.extend(probs.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


<ipython-input-28-1450b7217452>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_state.bin'))


Test loss: 0.148043567395964 | Test accuracy: 0.9591836734693877


In [29]:
print(classification_report(all_labels, all_preds))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96       335
           1       1.00      0.92      0.96       351

    accuracy                           0.96       686
   macro avg       0.96      0.96      0.96       686
weighted avg       0.96      0.96      0.96       686



In [30]:
# Calculate AUC
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(all_labels, all_probs)
print(f"AUC: {auc:.4f}")

AUC: 0.9921
